# SemEval-2020 Task 4: Commonsense Validation and Explanation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rlwPIErsvGcYYgM6s_FT9FDn9lcHzvVU?usp=sharing)

# 0. Getting started

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
%cd /content/gdrive/MyDrive/git/fer-tar

/content/gdrive/.shortcut-targets-by-id/1yfuNPQUT_G0CfNtdGxHhLi96giKyswbZ/git/fer-tar


## 0.1. Imports

In [43]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import scripts.data as data
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


## 0.2. Getting the data

In [44]:
X_train, X_dev, X_test, y_train, y_dev, y_test = data.load_data()

## 0.3. Preprocessing

In [45]:
X_ultimate_train, y_ultimate_train = data.melt_columns(X_train, y_train)
X_ultimate_dev, y_ultimate_dev = data.melt_columns(X_dev, y_dev)
X_ultimate_test, y_ultimate_test = data.melt_columns(X_test, y_test)

In [46]:
vec = TfidfVectorizer()
X_tfidf_train = vec.fit_transform(X_ultimate_train)
X_tfidf_test = vec.transform(X_ultimate_test)

# 1. Baseline

In [104]:
clf = LogisticRegression(C=0.02)
clf.fit(X_tfidf_train, y_ultimate_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=0.02, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [105]:
sent0_preds = clf.predict_proba(vec.transform(X_test['sent0']))[:, 0]
sent1_preds = clf.predict_proba(vec.transform(X_test['sent1']))[:, 0]
accuracy_score(y_test, sent0_preds > sent1_preds)

0.641